In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import Counter

# pd.set_option('display.max_colwidth', None)

In [25]:
df = pd.read_csv('steam_games.csv')

def wrapErrorDelete(path):
    try:
        os.remove(path)
    except:
        pass
        
wrapErrorDelete('steam_games_p1.csv')
wrapErrorDelete('steam_games_p2.csv')

In [26]:
df = df[df['name'].notna()]
df.drop(["discount_price", "publisher", "developer", "release_date", "desc_snippet", "types", "achievements", "recent_reviews", "mature_content"], axis=1, inplace=True)

In [27]:
for row in df.iterrows():
    if pd.isna(row[1][3]) and pd.isna(row[1][6]):
        print("deleting: ", row[1][1])
        df.drop(row[0], inplace=True)
        

deleting:  Ghost In The Shell: Stand Alone Complex 2nd Gig
deleting:  Ghost In The Shell: Stand Alone Complex Season 1
deleting:  Gintama Season 4
deleting:  CRYENGINE - Sample Assets
deleting:  Worms Reloaded: The "Pre-order Forts and Hats" DLC Pack
deleting:  Krater - Soundtrack
deleting:  Dustforce Soundtrack
deleting:  The Incredible Adventures of Van Helsing II - OST
deleting:  D4: SWERY's Choice Costume Set -4 Cups of Coffee-
deleting:  D4: SWERY's Choice Costume Set -4 Bottles of Tequila-
deleting:  Kemono Friends Season 1
deleting:  New Game Season 1
deleting:  Showa Genroku Rakugo Shinju Season 1
deleting:  Con Man Season 1
deleting:  DayZ Tools
deleting:  Forge of Gods: Ultimate Titan Bundle
deleting:  Killing Floor "London's Finest" Character Pack
deleting:  Dungeon Defenders Capture the Flag Pre-Alpha Pass (Free DLC)
deleting:  Digital Sound Factory: Session Drummer 3 Expansions
deleting:  Mob Psycho 100 Season 1
deleting:  Re:ZERO -Starting Life in Another World- Season 1


In [28]:
duplicated_items = df[df["name"].duplicated()]["name"].values.tolist()
games = {}
print(df.shape)
for row in df.iterrows():
    if row[1][1] in duplicated_items:
        if row[1][1] in games:
            games[row[1][1]].append(row[1].isna().sum())
        else:
            games[row[1][1]] = [row[1].isna().sum()]
for item in games:
    games[item].sort()
    count = 0
    for row in df.iterrows():
        if row[1][1] == item and (row[1].isnull().sum() == games[item][0]):
            df.drop(row[0], inplace=True)
            count += 1
            print("deleting: ", row[1][1])
        
        if count + 1 == len(games[item]):
            break
print(df.shape)
df.to_csv("steam_games_p1.csv", index=False)

(40675, 11)
deleting:  RUSH
deleting:  Project Highrise
deleting:  Batman: Arkham City - Game of the Year Edition
deleting:  Angels of Death
deleting:  Luna
deleting:  Flashback
deleting:  New York Bus Simulator
deleting:  Warhammer Quest
deleting:  City Builder
deleting:  Killing Time
deleting:  Jigsaw Puzzles
deleting:  Hide and Seek
deleting:  Hidden Object Adventure Bundle
deleting:  Arcade Bundle
deleting:  Arcade Bundle
deleting:  Soundtrack Edition
deleting:  Solitaire Collection
deleting:  Ultimate Arena
deleting:  Black Rose
deleting:  Solstice
deleting:  Surge
deleting:  Game & Soundtrack
deleting:  7 Days in Dream 光尘
deleting:  Bounce
deleting:  Game + Soundtrack
deleting:  Chaos Theory
deleting:  Deluxe Edition
deleting:  Deluxe Edition
deleting:  Rumpus
deleting:  Space Maze
deleting:  Spark
deleting:  The Mine
deleting:  Memories
deleting:  Ashes
deleting:  Dodge
deleting:  We Were Here Too: Supporter Edition
deleting:  The Deer
deleting:  Zombie Apocalypse
deleting:  Aft

In [48]:
df = pd.read_csv('steam_games_p1.csv')
all_review_cols = ["all_review_sentiment", "all_review_sentiment_num", "all_review_num"]
all_review_df = pd.DataFrame(columns=all_review_cols)
idx = 0
for row in df.iterrows():
    if row[1][2] == None or pd.isna(row[1][2]):
        all_review_df.loc[idx] = ["None", 0, 0]
    else:
        try:
            items = row[1][2].split('(')
            sentiment = items[0].lower().strip()[0:-1]
            items = items[1].split(')')
            total = int(items[0].strip().replace(',', ''))
            percent = int(items[1].split('%')[0].strip()[3:]) / 100
            all_review_df.loc[idx] = [sentiment, int(total * percent), total]
        except:
            all_review_df.loc[idx] = ["None", 0, 0]
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
df.drop(["all_reviews"], axis=1, inplace=True)
df = pd.concat([df, all_review_df], axis=1)
df['all_review_sentiment'] = df['all_review_sentiment'].astype('category').cat.codes
df.to_csv("steam_games_p2.csv", index=False)

12.271790341578328 %
24.541126815861798 %
36.810463290145265 %
49.07979976442874 %
61.34913623871221 %
73.61847271299568 %
85.88780918727915 %
98.15714566156262 %


In [68]:
df = pd.read_csv('steam_games_p2.csv')
df['popular_tags'].fillna(df['genre'], inplace=True)
df['genre'].fillna(df['popular_tags'], inplace=True)
popular_tags = set()
pt_list = []
genre = set()
genre_list = []
for i in df['popular_tags'].to_list():
    temp = i.split(',')
    pt_list.extend(temp)
    for j in temp:
        popular_tags.add(j.lower().strip())
popular_tags = list(popular_tags)
for i in df['genre'].to_list():
    temp = i.split(',')
    genre_list.extend(temp)
    for j in temp:
        genre.add(j.lower().strip())
genre = list(genre)
pd.Series(pt_list).to_csv('popular_tags_freq.csv', index=False)
pd.Series(genre_list).to_csv('genre_freq.csv', index=False)